In [14]:
num='112'
i=0
window=num[i]+num[i+1]+num[i+2]
num[i]==num[i+1]==num[i+2]

False

In [12]:
(n & (n - 1)) == 0 and (n - 1) % 3 == 0

True

In [4]:
(0/4)%4==0

True

In [ ]:
def largestGoodInteger(num: str) -> str:
    result = ''
    for i in range(len(num) - 2):
        window = num[i:i+3]
        if window[0] == window[1] == window[2]:
            if result == '' or int(window) > int(result):
                result = window
    return result
        

In [23]:
class Solution:
    def isPowerOfThree(self, n: int) -> bool:
        if n <= 0:
            return False
        while n % 3 == 0:
            n //= 3
        return n == 1
                

'999'

In [27]:
import math

def is_power_of_three(n: int) -> bool:
    return n > 0 and 3 ** round(math.log(n, 3)) == n


2

In [1]:
import requests
url = "http://localhost:8000/add"  # Replace with your endpoint URL
data = {
    'a':100,
    'b':100
}
headers={
    'accept':'application/json',
    'Content-Type':'application/json'
}

response = requests.post(url, json=data,headers=headers)

In [3]:
response.text

'{"result":200.0}'

In [4]:
ip1={
    
            "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to add"},
            "b": {"type": "integer", "description": "Second integer to add"}
            },
            "required": ["a", "b"]
        }

In [5]:
ip1==input_scheme[0]

True

In [1]:
input_scheme=[
    {
        "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to add"},
            "b": {"type": "integer", "description": "Second integer to add"}
            },
            "required": ["a", "b"]
    },
    {
        "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to Multiply"},
            "b": {"type": "float", "description": "Second integer to Multiply"}
            },
        
            "required": ["a", "b"]
        
    }
]

In [ ]:
type(input_scheme[1])


dict